In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from pathlib import Path

import pandas as pd
import numpy as np

In [3]:
cd ..

C:\Projects\python\recommender


In [4]:
import torch as T
import torch.nn as nn
import torch.optim as optim

from datasets import MovelenDataset, TorchMovielen10k

## Init Dataloader instance

In [5]:
DEVICE = T.device('cpu')
BATCH = 32
SHUFFLE = False
WORKERS = 0
DATA_PATH = Path("./inputs/ml-100k/u.data")
ITEM_PATH = Path("./inputs/ml-100k/u.item")

In [7]:
databunch = TorchMovielen10k(DATA_PATH, item_path=ITEM_PATH, user_min=4, item_min=4)
databunch.batch(BATCH)
databunch.device(DEVICE)
databunch.shuffle(SHUFFLE)
databunch.workers(WORKERS)

2019-09-08 13:40:37,331 - C:\Projects\python\recommender\utils.py - INFO - Read dataset in inputs\ml-100k\u.data
I0908 13:40:37.331149 11664 torch_movielen.py:101] Read dataset in inputs\ml-100k\u.data
2019-09-08 13:40:37,349 - C:\Projects\python\recommender\utils.py - INFO - Original user size: 943
I0908 13:40:37.349512 11664 torch_movielen.py:105] Original user size: 943
2019-09-08 13:40:37,352 - C:\Projects\python\recommender\utils.py - INFO - Original item size: 1682
I0908 13:40:37.352475 11664 torch_movielen.py:106] Original item size: 1682
2019-09-08 13:40:37,357 - C:\Projects\python\recommender\utils.py - INFO - Filter user size: 943
I0908 13:40:37.357462 11664 torch_movielen.py:112] Filter user size: 943
2019-09-08 13:40:37,360 - C:\Projects\python\recommender\utils.py - INFO - Filter item size: 1413
I0908 13:40:37.360455 11664 torch_movielen.py:113] Filter item size: 1413
c:\projects\python\recommender\.venv\lib\site-packages\pandas\core\generic.py:8682: SettingWithCopyWarning

In [8]:
train_dl = databunch.get_dataloader(dataset_type='train')
train_it = iter(train_dl)

In [9]:
user_index, pos_feats, neg_feats = train_it.next()
print("positive batch shape:", pos_feats.shape)
print("negtive batch shape:", neg_feats.shape)
print("user index shape:", user_index.shape)

positive batch shape: torch.Size([32, 3807])
negtive batch shape: torch.Size([32, 3807])
user index shape: torch.Size([32])


## Test each fm model

In [10]:
feat_dim = databunch.feat_dim
num_dim = 32
init_mean = 1

In [11]:
feat_dim

3807

In [12]:
from models import TorchFM, TorchHrmFM, TorchPrmeFM, TorchTransFM

### Test FM Model

In [13]:
fm_model = TorchFM(feat_dim, num_dim, init_mean)
fm_model

TorchFM()

In [14]:
user_index, pos_feats, neg_feats = train_it.next()

In [15]:
fm_model(pos_feats, neg_feats)

(tensor([-64.1903, -43.7776, -39.0514, -59.5614, -12.4093, -49.2389, -43.8010,
         -20.8921, -42.6683, -39.1546, -33.4211, -42.0648, -12.9598, -21.9878,
         -28.9777,  -3.6385, -22.9925,  39.6334, -51.3753,  -9.1928, -17.1664,
         -36.8083, -23.3391, -31.5518,  39.8880, -31.3575, -13.6581, -33.2623,
         -36.0992, -60.2210, -69.1044, -30.0486], dtype=torch.float64,
        grad_fn=<SqueezeBackward0>),
 tensor([ 1.5140, -0.8733,  2.8072,  3.2439,  1.7383,  0.9903,  1.2133,  3.6962,
         -0.2099,  0.2052, -0.7542,  0.1657,  3.4462,  0.2641, -1.1066, -2.0452,
          0.5183,  0.3534,  0.3481,  0.8460,  1.1338, -0.7853, -0.3255,  0.3606,
          3.3211, -0.1820,  1.3389, -0.2750, -0.5071, -0.0322,  1.2707,  1.5460],
        dtype=torch.float64, grad_fn=<SqueezeBackward0>))

### Test HRM FM model

In [16]:
hrm_model = TorchHrmFM(feat_dim, num_dim, init_mean)
hrm_model

TorchHrmFM()

In [17]:
hrm_model(pos_feats, neg_feats)

(tensor([271.7594, 384.6150, 387.4170, 261.1064, 296.9858, 367.6979, 253.7847,
         215.1222, 388.3259, 666.9026, 458.0549, 277.8743, 645.2322, 156.9256,
         161.5078, 179.2077, 169.0403, 324.6285, 223.6107, 488.6003, 308.5906,
         141.7886, 108.3088, 210.6180, 309.3110, 102.9862, 146.6035, 212.3038,
         297.4553, 360.9046, 350.7762, 128.4124], dtype=torch.float64,
        grad_fn=<SqueezeBackward0>),
 tensor([261.8470, 447.6939, 235.7539, 271.7186, 155.7334, 329.8108, 157.5224,
         152.1141, 223.8814, 226.9145, 360.7292, 280.6197, 366.8122,  97.5011,
         231.6856, 134.6230, 226.9655, 169.0502, 102.2834, 367.0369, 293.3065,
         210.5617, 301.1654, 203.2470, 308.7432, 152.0882, 209.9438, 218.6886,
         322.0774, 220.5088, 197.0379, 212.8642], dtype=torch.float64,
        grad_fn=<SqueezeBackward0>))

In [18]:
item = T.sparse.sum(pos_feats, dim=1)
item

tensor(indices=tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,
                        14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
                        28, 29, 30, 31]]),
       values=tensor([ 8.,  9.,  9.,  8.,  8.,  9.,  7.,  7.,  9., 12., 10.,
                       8., 12.,  6.,  6.,  6.,  6.,  8.,  7., 10.,  8.,  6.,
                       5.,  7.,  8.,  5.,  6.,  7.,  8.,  9.,  9.,  6.]),
       size=(32,), nnz=32, dtype=torch.float64, layout=torch.sparse_coo)

In [19]:
item.to_dense().unsqueeze(dim=1).shape

torch.Size([32, 1])

### Test PRME FM Model

In [20]:
prme_model = TorchPrmeFM(feat_dim, num_dim, init_mean)
prme_model

TorchPrmeFM()

In [21]:
prme_model(pos_feats, neg_feats)

(tensor([ 624.5587,  808.3335,  685.4854,  555.8014,  596.5832,  759.5734,
          456.4375,  462.9926,  700.5415, 1342.8371,  999.2225,  639.3323,
         1375.1834,  379.3050,  307.9397,  331.3744,  309.2965,  640.0821,
          483.5413,  953.9946,  648.8265,  339.4598,  217.3180,  433.8903,
          540.2731,  248.5719,  265.5579,  407.6169,  638.7647,  765.2503,
          729.8961,  311.3912], dtype=torch.float64, grad_fn=<SqueezeBackward0>),
 tensor([589.1098, 947.7146, 415.0550, 567.4627, 316.2383, 776.1557, 325.8328,
         302.5159, 434.6137, 434.0046, 774.6679, 611.3691, 770.9170, 206.1869,
         446.3521, 322.7631, 464.3196, 360.0030, 246.4301, 736.5838, 646.3014,
         513.9927, 624.2010, 450.5826, 589.2988, 346.5698, 353.5650, 452.5794,
         607.8989, 461.9401, 434.6386, 445.3938], dtype=torch.float64,
        grad_fn=<SqueezeBackward0>))

### Test Trans FM Model

In [22]:
trans_model = TorchTransFM(feat_dim, num_dim, init_mean)

In [23]:
trans_model(pos_feats, neg_feats)

(tensor([ 877.6922, 1284.8854, 1169.6049,  848.1012,  911.5739, 1186.9655,
          622.3278,  690.7450, 1224.0787, 2181.0642, 1542.0397,  869.9841,
         2114.0940,  550.9422,  500.9470,  495.6111,  481.7289, 1047.6194,
          737.7010, 1448.4945,  954.8884,  517.7452,  356.1667,  659.3029,
          981.3259,  323.5962,  480.6206,  621.6405,  936.6802, 1142.2824,
         1101.6843,  498.1779], dtype=torch.float64, grad_fn=<SqueezeBackward0>),
 tensor([ 903.9866, 1587.0843,  697.0481,  897.3490,  586.8591, 1229.5051,
          530.8814,  552.6464,  810.6351,  895.0207, 1292.9848,  965.0210,
         1279.7142,  355.9132,  725.0196,  498.4545,  672.3376,  696.3893,
          412.9733, 1187.7440, 1002.4424,  646.9900,  817.7635,  727.2876,
          847.4854,  474.3863,  593.1399,  625.8628, 1002.5214,  750.8584,
          706.0431,  629.1199], dtype=torch.float64, grad_fn=<SqueezeBackward0>))

#### Find size error!!

#### Test trans model forward

In [24]:
feature_dim = trans_model._feature_dim
param_linear = trans_model.param_linear
param_emb = trans_model.param_emb
param_trans = trans_model.param_trans

In [25]:
pos_linear = T.mm(pos_feats, param_linear)
neg_linear = T.mm(neg_feats, param_linear)
print(pos_linear.shape)
print(neg_linear.shape)

torch.Size([32, 1])
torch.Size([32, 1])


In [26]:
var_sum_op = T.ones(feature_dim, 1, dtype=T.double)
var_emb_product = T.sum(T.pow(param_emb, 2), dim=1, keepdim=True)
var_trans_product = T.sum(T.pow(param_trans, 2), dim=1, keepdim=True)
var_emb_trans_product = T.sum(param_emb * param_trans,
                              dim=1,
                              keepdim=True)
print(var_sum_op.shape)
print(var_emb_product.shape)
print(var_trans_product.shape)
print(var_emb_trans_product.shape)

torch.Size([3807, 1])
torch.Size([3807, 1])
torch.Size([3807, 1])
torch.Size([3807, 1])


In [27]:
# Common term positive
pos_feats_sum = T.mm(pos_feats, var_sum_op)
pos_emb_mul = T.mm(pos_feats, param_emb)
pos_trans_mul = T.mm(pos_feats, param_trans)

# Common terms negative
neg_feats_sum = T.mm(neg_feats, var_sum_op)
neg_emb_mul = T.mm(neg_feats, param_emb)
neg_trans_mul = T.mm(neg_feats, param_trans)

In [28]:
print(pos_feats_sum.shape)
print(pos_emb_mul.shape)
print(pos_trans_mul.shape)
print("---------------------------")
print(neg_feats_sum.shape)
print(neg_emb_mul.shape)
print(neg_trans_mul.shape)

torch.Size([32, 1])
torch.Size([32, 32])
torch.Size([32, 32])
---------------------------
torch.Size([32, 1])
torch.Size([32, 32])
torch.Size([32, 32])


In [29]:
# Term 1 pos
prod_term_pos = T.mm(pos_feats, var_emb_product)
term_1_pos = prod_term_pos * pos_feats_sum

# Term 1 neg
prod_term_neg = T.mm(neg_feats, var_emb_product)
term_1_neg = prod_term_neg * neg_feats_sum

In [30]:
print(prod_term_pos.shape)
print(term_1_pos.shape)
print("-----------------------")
print(prod_term_neg.shape)
print(term_1_neg.shape)

torch.Size([32, 1])
torch.Size([32, 1])
-----------------------
torch.Size([32, 1])
torch.Size([32, 1])


In [31]:
# Term 2 pos
prod_term_pos = T.mm(pos_feats, param_trans)
term_2_pos = prod_term_pos * pos_feats_sum

# Term 2 neg
prod_term_neg = T.mm(neg_feats, var_trans_product)
term_2_neg = prod_term_neg * pos_feats_sum

In [32]:
print(term_2_pos.shape)
print(term_2_neg.shape)

torch.Size([32, 32])
torch.Size([32, 1])


#### Find error !!!!

In [33]:
trans_model(pos_feats, neg_feats)

(tensor([ 877.6922, 1284.8854, 1169.6049,  848.1012,  911.5739, 1186.9655,
          622.3278,  690.7450, 1224.0787, 2181.0642, 1542.0397,  869.9841,
         2114.0940,  550.9422,  500.9470,  495.6111,  481.7289, 1047.6194,
          737.7010, 1448.4945,  954.8884,  517.7452,  356.1667,  659.3029,
          981.3259,  323.5962,  480.6206,  621.6405,  936.6802, 1142.2824,
         1101.6843,  498.1779], dtype=torch.float64, grad_fn=<SqueezeBackward0>),
 tensor([ 903.9866, 1587.0843,  697.0481,  897.3490,  586.8591, 1229.5051,
          530.8814,  552.6464,  810.6351,  895.0207, 1292.9848,  965.0210,
         1279.7142,  355.9132,  725.0196,  498.4545,  672.3376,  696.3893,
          412.9733, 1187.7440, 1002.4424,  646.9900,  817.7635,  727.2876,
          847.4854,  474.3863,  593.1399,  625.8628, 1002.5214,  750.8584,
          706.0431,  629.1199], dtype=torch.float64, grad_fn=<SqueezeBackward0>))